# Using Neo4j graph data science package

In [1]:
from graphdatascience import GraphDataScience
import pandas as pd
import os
from pprint import pprint 

In [2]:
NEO4J_URI = "bolt://localhost:7687"
gds = GraphDataScience(NEO4J_URI, auth=("neo4j","123"))

# connect with AuraDS
# Configure the driver with AuraDS-recommended settings
# gds = GraphDataScience("neo4j+s://my-aura-ds.databases.neo4j.io:7687", auth=("neo4j", "my-password"), aura_ds=True)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('advertisedListenAddress' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('serverLocation' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'


In [3]:
print(gds.version())

2.7.0


In [4]:
tech_df = pd.read_excel(r"C:\\Users\25335\Downloads\neo4j_astoria\data\Technology Skills.xlsx")
skills_df = pd.read_excel(r"C:\Users\25335\Downloads\neo4j_astoria\data\Skills.xlsx")
knowledge_df = pd.read_excel(r"C:\Users\25335\Downloads\neo4j_astoria\data\Knowledge.xlsx")
related_title_df = pd.read_excel(r"C:\Users\25335\Downloads\neo4j_astoria\data\Related Occupations.xlsx")

skills_df.rename(columns = {'Element Name':'Skill'}, inplace = True)
knowledge_df.rename(columns = {'Element Name':'Knowledge'}, inplace = True)

In [5]:
tech_example_listed = tech_df.groupby(["Title"])["Example"].apply(lambda x: [item.lower() for item in list(set(x))]).reset_index("Title")
tech_HOT_listed = tech_df.groupby(["Title"])["Hot Technology"].apply(list).reset_index("Title")

data_value_threshhold = skills_df["Data Value"] > 4.5
skill_listed = skills_df[data_value_threshhold].groupby(["Title"])["Skill"].apply(lambda x: [item.lower() for item in list(set(x))]).reset_index("Title")
knowledge_listed = knowledge_df[data_value_threshhold].groupby(["Title"])["Knowledge"].apply(lambda x: [item.lower() for item in list(set(x))]).reset_index("Title")
related_listed = related_title_df.groupby(["Title"])["Related Title"].apply(lambda x: [item.lower() for item in list(set(x))]).reset_index("Title")

merged_df = tech_example_listed.merge(tech_HOT_listed, on="Title", how="left")
merged_df = merged_df.merge(skill_listed, on="Title", how="left")
merged_df = merged_df.merge(knowledge_listed, on="Title", how="left")
merged_df = merged_df.merge(related_listed, on="Title", how="left")

merged_df.fillna("NA", inplace=True)

C:\Users\25335\AppData\Local\Temp\ipykernel_2168\3533319999.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  knowledge_listed = knowledge_df[data_value_threshhold].groupby(["Title"])["Knowledge"].apply(lambda x: [item.lower() for item in list(set(x))]).reset_index("Title")


In [6]:
# some more preprocessing
merged_df = merged_df.drop_duplicates(subset='Title', keep='first')
merged_df = merged_df.drop_duplicates(subset='Related Title', keep='first')
merged_df['Title'] = merged_df['Title'].str.lower()

merged_df.sample(5)

,Title,Example,Hot Technology,Skill,Knowledge,Related Title
186,correspondence clerks,"[data entry software, sap, microsoft word, mic...","[Y, N, N, Y, Y, Y, Y, Y, Y]",NA,NA,"[interviewers, except eligibility and loan, re..."
412,helpers--electricians,"[recordkeeping software, computer-aided drafti...","[Y, Y, Y, N, N]",NA,NA,"[boilermakers, electricians, electrical power-..."
221,dermatologists,"[ge healthcare centricity practice solution, e...","[N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, ...",[reading comprehension],[clerical],"[cardiologists, family medicine physicians, em..."
721,recreational vehicle service technicians,"[email software, microsoft word, inventory tra...","[N, N, Y, Y, N, N, N]",NA,NA,"[mobile heavy equipment mechanics, except engi..."
737,retail salespersons,[advanced retail management systems retail pro...,"[Y, Y, N, Y, Y, Y, N, N, Y, N, N, Y, N, N, N, ...",NA,NA,"[door-to-door sales workers, news and street v..."


In [7]:
for index, row in merged_df.iterrows():

        tech_list = row["Example"]
        skill_list = row["Skill"]
        knowledge_list = row["Knowledge"]
        hot_tech_list = row["Hot Technology"]
        related_list = row["Related Title"]

        if tech_list is not None:

          for i in range(len(tech_list)):

            #add tech skills

            row_level_query1 = """
            MERGE (title:Title {name: $title_name})

            MERGE (tech:TechSkill {name: $tech_name, hot: $hot_bool})

            MERGE (title)-[:HAS_TECH_SKILL]->(tech)
            """
            gds.run_cypher(row_level_query1, 
                           params={"title_name":row["Title"],
                                   "tech_name":tech_list[i],
                                   "hot_bool":hot_tech_list[i]               
            })

        #add skills and knowledge
        if skill_list != "NA" and skill_list is not None:

          for j in range(len(skill_list)):
            row_level_query2 = """
            MERGE (title:Title {name: $title_name})
            MERGE (skill:Skill {name: $skill_name})
            MERGE (title)-[:HAS_SKILL]->(skill)
            """
            gds.run_cypher(row_level_query2, 
                           params={"title_name":row["Title"],
                                  "skill_name":tech_list[j]               
            })


        # add knowledge and skill
        if knowledge_list != "NA" and knowledge_list is not None:
          for k in range(len(knowledge_list)):
            #add skills and knowledge
            row_level_query3 = """
            MERGE (title:Title {name: $title_name})
            MERGE (knowledge:Knowledge {name: $knowledge_name})
            MERGE (title)-[:HAS_KNOWLEDGE]->(knowledge)
        """
            gds.run_cypher(row_level_query3, 
                           params={"title_name":row["Title"],
                                  "knowledge_name":knowledge_list[k]              
            })

        #add related occupations
        if related_list != "NA" and related_list is not None:
          for l in range(len(related_list)):
            row_level_query4 = """
            MERGE (title:Title {name: $title_name})
            MERGE (related:Title {name: $related_name})
            MERGE (title)-[:HAS_RELATION]-(related)
            """
            gds.run_cypher(row_level_query4, 
                           params={"title_name":row["Title"],
                                  "related_name":related_list[l]              
            })

        print(f"{index} out of {len(merged_df)} DONE")

0 out of 902 DONE
1 out of 902 DONE
2 out of 902 DONE
3 out of 902 DONE
4 out of 902 DONE
5 out of 902 DONE
6 out of 902 DONE
7 out of 902 DONE
8 out of 902 DONE
9 out of 902 DONE
10 out of 902 DONE
11 out of 902 DONE
12 out of 902 DONE
13 out of 902 DONE
14 out of 902 DONE
15 out of 902 DONE
16 out of 902 DONE
17 out of 902 DONE
18 out of 902 DONE
19 out of 902 DONE
20 out of 902 DONE
21 out of 902 DONE
22 out of 902 DONE
23 out of 902 DONE
24 out of 902 DONE
25 out of 902 DONE
26 out of 902 DONE
27 out of 902 DONE
28 out of 902 DONE
29 out of 902 DONE
30 out of 902 DONE
31 out of 902 DONE
32 out of 902 DONE
33 out of 902 DONE
34 out of 902 DONE
35 out of 902 DONE
36 out of 902 DONE
37 out of 902 DONE
38 out of 902 DONE
39 out of 902 DONE
40 out of 902 DONE
41 out of 902 DONE
42 out of 902 DONE
43 out of 902 DONE
44 out of 902 DONE
45 out of 902 DONE
46 out of 902 DONE
47 out of 902 DONE
48 out of 902 DONE
49 out of 902 DONE
50 out of 902 DONE
51 out of 902 DONE
52 out of 902 DONE
53 

## Centrality measure

In [8]:
G, result = gds.graph.project("relation_test", "Title", "HAS_RELATION")

print(f"The projection took {result['projectMillis']} ms")

# We can use convenience methods on `G` to check if the projection looks correct
print(f"Graph '{G.name()}' node count: {G.node_count()}")
print(f"Graph '{G.name()}' node labels: {G.node_labels()}")
print(f"Graph '{G.name()}' relationship count: {G.relationship_count()}")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.list' is deprecated.)} {position: line: 1, column: 40, offset: 39} for query: 'CALL gds.graph.list($graph_name) YIELD schema, database'


The projection took 24 ms
Graph 'relation_test' node count: 923
Graph 'relation_test' node labels: ['Title']
Graph 'relation_test' relationship count: 13080


In [9]:
eigenvector_centrality_result = gds.eigenvector.mutate(G, maxIterations=100, mutateProperty="eigenvectorCentrality")

In [10]:
eigenvector_centrality_result

mutateMillis                                                              0
nodePropertiesWritten                                                   923
ranIterations                                                           100
didConverge                                                           False
centralityDistribution    {'min': 0.0, 'max': 0.46537971496582026, 'p90'...
postProcessingMillis                                                    142
preProcessingMillis                                                       0
computeMillis                                                           106
configuration             {'mutateProperty': 'eigenvectorCentrality', 'j...
Name: 0, dtype: object

In [11]:
eigenvector_centrality_result.centralityDistribution

{'min': 0.0,
 'max': 0.46537971496582026,
 'p90': 0.01036310195922674,
 'p999': 0.46537971496581854,
 'p99': 0.19938755035400213,
 'p50': 0.0001336019486171125,
 'p75': 0.001373641192911279,
 'p95': 0.032069683074949396,
 'mean': 0.007387174331791837}

In [12]:
gds.graph.nodeProperties.write(G, ["eigenvectorCentrality"])

writeMillis                               59
graphName                      relation_test
nodeProperties       [eigenvectorCentrality]
propertiesWritten                        923
Name: 0, dtype: object

In [13]:
pd.set_option('display.max_colwidth', None)

def display_top_20(centrality_measure):
    """
    Function to execute the Cypher query to retrieve the top 20 cities with the highest centrality measure.
    """
    query = f"""
    MATCH (n:Title)
    WHERE n.{centrality_measure} IS NOT NULL
    RETURN n.name AS name, n.{centrality_measure} AS {centrality_measure}
    ORDER BY n.{centrality_measure} DESC
    LIMIT 20
    """
    result = gds.run_cypher(query)
    return result

    # Display the result
    #for res in result:
        #pprint(res["name"])


display_top_20("eigenvectorCentrality").head(20)

,name,eigenvectorCentrality
0,"woodworking machine setters, operators, and tenders, except sawing",0.465380
1,"rolling machine setters, operators, and tenders, metal and plastic",0.288922
2,"tool grinders, filers, and sharpeners",0.282746
3,industrial machinery mechanics,0.255133
4,"welding, soldering, and brazing machine setters, operators, and tenders",0.239907
5,tool and die makers,0.232556
6,"grinding and polishing workers, hand",0.213068
7,"multiple machine tool setters, operators, and tenders, metal and plastic",0.208561
8,"paper goods machine setters, operators, and tenders",0.207760
9,machine feeders and offbearers,0.199387


## community detection

In [14]:
# view graph data in pandas
# gds.graph.list()

### weakly connected components

In [15]:

result = gds.wcc.mutate(G, mutateProperty="componentId")

print(f"Components found: {result.componentCount}")

Components found: 1


In [17]:
query = """
    CALL gds.graph.nodeProperties.stream('relation_test', 'componentId')
    YIELD nodeId, propertyValue
    WITH gds.util.asNode(nodeId).name AS node, propertyValue AS componentId
    WITH componentId, collect(node) AS group
    WITH componentId, group, size(group) AS componentSize
    RETURN componentId, componentSize, group
    ORDER BY componentSize DESC
"""

components = gds.run_cypher(query)
components

,componentId,componentSize,group
0,0,923,"[accountants and auditors, billing and posting clerks, sales representatives of services, except advertising, insurance, financial services, and travel, financial and investment analysts, statistical assistants, credit authorizers, checkers, and clerks, compensation, benefits, and job analysis specialists, credit analysts, management analysts, financial managers, tax preparers, personal financial advisors, budget analysts, payroll and timekeeping clerks, financial examiners, first-line supervisors of office and administrative support workers, brokerage clerks, tax examiners and collectors, and revenue agents, financial risk specialists, treasurers and controllers, bookkeeping, accounting, and auditing clerks, actors, writers and authors, news analysts, reporters, and journalists, agents and business managers of artists, performers, and athletes, dancers, costume attendants, models, art, drama, and music teachers, postsecondary, poets, lyricists and creative writers, special effects artists and animators, self-enrichment teachers, art directors, music directors and composers, makeup artists, theatrical and performance, producers and directors, choreographers, fine artists, including painters, sculptors, and illustrators, film and video editors, musicians and singers, broadcast announcers and radio disc jockeys, talent directors, actuaries, financial quantitative analysts, investment fund managers, insurance underwriters, compensation and benefits managers, business intelligence analysts, insurance sales agents, securities, commodities, and financial services sales agents, economists, acupuncturists, cardiologists, family medicine physicians, emergency medicine physicians, pediatricians, general, advanced practice psychiatric nurses, neurologists, orthopedic surgeons, except pediatric, chiropractors, physical medicine and rehabilitation physicians, dermatologists, massage therapists, general internal medicine physicians, psychiatrists, naturopathic physicians, recreational therapists, urologists, allergists and immunologists, pediatric surgeons, obstetricians and gynecologists, nurse practitioners, acute care nurses, paramedics, nursing assistants, respiratory therapists, registered nurses, emergency medical technicians, anesthesiologists, licensed practical and licensed vocational nurses, nurse midwives, clinical nurse specialists, physical therapists, critical care nurses, physician assistants, adapted physical education specialists, health specialties teachers, postsecondary, kindergarten teachers, except special education, special education teachers, elementary school, middle school teachers, except special and career/technical education, career/technical education teachers, postsecondary, recreation and fitness studies teachers, postsecondary, career/technical education teachers, middle school, adult basic education, adult secondary education, and english as a second language instructors, special education teachers, secondary school, career/technical education teachers, secondary school, special education teachers, preschool, school psychologists, teaching assistants, special education, secondary school teachers, except special and career/technical education, ...]"


In [18]:
largest_component = components["componentId"][0]

print(f"The largest component has the id {largest_component} with {components['componentSize'][0]} techs.")

The largest component has the id 0 with 923 techs.


In [19]:

largest_component_graph, _ = gds.beta.graph.project.subgraph(
    "largest_connected_components", G, f"n.componentId={largest_component}", "*"
)
largest_component_graph

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('gds.beta.graph.project.subgraph' has been replaced by 'gds.graph.filter')} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.beta.graph.project.subgraph($graph_name, $from_graph_name, $node_filter, $relationship_filter, $config)'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.list' is deprecated.)} {position: line: 1, column: 106, offset: 105} for query: 'CALL gds.graph.list($graph_name) YIELD graphName, nodeCount, relatio

Graph({'graphName': 'largest_connected_components', 'nodeCount': 923, 'relationshipCount': 13080, 'database': 'neo4j', 'configuration': {'relationshipProperties': {}, 'jobId': '553d9758-a18e-4035-aea3-b117cc58ec44', 'validateRelationships': False, 'nodeFilter': 'n.componentId=0', 'sudo': False, 'relationshipProjection': {'HAS_RELATION': {'aggregation': 'DEFAULT', 'orientation': 'NATURAL', 'indexInverse': False, 'properties': {}, 'type': 'HAS_RELATION'}}, 'readConcurrency': 4, 'nodeProperties': {}, 'nodeProjection': {'Title': {'label': 'Title', 'properties': {}}}, 'logProgress': True, 'concurrency': 4, 'creationTime': neo4j.time.DateTime(2024, 7, 7, 2, 20, 52, 295717700, tzinfo=<UTC>), 'relationshipFilter': '*', 'parameters': {}}, 'schema': {'graphProperties': {}, 'nodes': {'Title': {'eigenvectorCentrality': 'Float (DefaultValue(NaN), TRANSIENT)', 'componentId': 'Integer (DefaultValue(-9223372036854775808), TRANSIENT)'}}, 'relationships': {'HAS_RELATION': {}}}, 'memoryUsage': '437 KiB'}

### louvain

In [20]:
gds.louvain.mutate(largest_component_graph, mutateProperty="louvainCommunityId")

mutateMillis                                                                                                                                                                                                                                                                                                                                                               0
nodePropertiesWritten                                                                                                                                                                                                                                                                                                                                                    923
modularity                                                                                                                                                                                                                                                                    

In [21]:
gds.graph.nodeProperties.write(largest_component_graph, ["louvainCommunityId"])

writeMillis                                    82
graphName            largest_connected_components
nodeProperties               [louvainCommunityId]
propertiesWritten                             923
Name: 0, dtype: object

In [22]:
# check names in certain communities
gds.run_cypher(
    """
    MATCH (n) WHERE 'louvainCommunityId' IN keys(n) 
    RETURN n.name, n.louvainCommunityId LIMIT 10
    """
)

,n.name,n.louvainCommunityId
0,accountants and auditors,689
1,billing and posting clerks,689
2,"sales representatives of services, except advertising, insurance, financial services, and travel",689
3,financial and investment analysts,689
4,statistical assistants,798
5,"credit authorizers, checkers, and clerks",689
6,"compensation, benefits, and job analysis specialists",689
7,credit analysts,689
8,management analysts,798
9,financial managers,689


In [23]:
query = """
    CALL gds.graph.nodeProperties.stream('largest_connected_components', 'louvainCommunityId')
    YIELD nodeId, propertyValue
    WITH gds.util.asNode(nodeId).name AS node, propertyValue AS communityId
    WITH communityId, collect(node) AS title_community
    WITH communityId, title_community, size(title_community) AS communitySize
    RETURN communityId, communitySize, title_community
    ORDER BY communitySize DESC
"""

communities = gds.run_cypher(query)
communities

communityId  communitySize  \
0          301            264   
1          689            197   
2          469            155   
3          914            152   
4          675             83   
5          798             72   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [24]:
G.drop()
gds.graph.drop('relation_test')

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.graph.drop($graph_name, $failIfMissing)'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.graph.drop($graph_name, $fail_if_missing, $db_name)'


## similarity: FastRP(node embedding) + KNN

In [25]:
# We define how we want to project our database into GDS
node_projection = ["Title", "TechSkill"]
relationship_projection = {"HAS_TECH_SKILL": {"orientation": "UNDIRECTED"}}

# Before actually going through with the projection, let's check how much memory is required
result = gds.graph.project.estimate(node_projection, relationship_projection)

print(f"Required memory for native loading: {result['requiredMemory']}")

Required memory for native loading: 1462 KiB


In [26]:
G.drop()
gds.graph.drop('TechSkill')

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.graph.drop($graph_name, $failIfMissing)'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.graph.drop($graph_name, $fail_if_missing, $db_name)'


In [27]:
G, result = gds.graph.project("TechSkill", node_projection, relationship_projection)

print(f"The projection took {result['projectMillis']} ms")

# We can use convenience methods on `G` to check if the projection looks correct
print(f"Graph '{G.name()}' node count: {G.node_count()}")
print(f"Graph '{G.name()}' node labels: {G.node_labels()}")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.list' is deprecated.)} {position: line: 1, column: 40, offset: 39} for query: 'CALL gds.graph.list($graph_name) YIELD schema, database'


The projection took 39 ms
Graph 'TechSkill' node count: 11032
Graph 'TechSkill' node labels: ['TechSkill', 'Title']


In [28]:
# We can also estimate memory of running algorithms like FastRP, so let's do that first
result = gds.fastRP.mutate.estimate(
    G,
    mutateProperty="embedding",
    randomSeed=42,
    embeddingDimension=20,
    iterationWeights=[0.8, 1, 1, 1],
)

print(f"Required memory for running FastRP: {result['requiredMemory']}")

Required memory for running FastRP: 3620 KiB


In [29]:
# Now let's run FastRP and mutate our projected graph 'purchases' with the results
result = gds.fastRP.mutate(
    G,
    mutateProperty="embedding",
    randomSeed=50,
    embeddingDimension=75,
    iterationWeights=[0.8, 1, 1, 1],
)

# Let's make sure we got an embedding for each node
print(f"Number of embedding vectors produced: {result['nodePropertiesWritten']}")

Number of embedding vectors produced: 11032


In [30]:
# Run kNN and write back to db (we skip memory estimation this time...)
result = gds.knn.write(
    G,
    topK=2,
    nodeProperties=["embedding"],
    randomSeed=42,
    concurrency=1,
    sampleRate=1.0,
    deltaThreshold=0.0,
    writeRelationshipType="SIMILAR",
    writeProperty="score",
)

print(f"Relationships produced: {result['relationshipsWritten']}")
print(f"Nodes compared: {result['nodesCompared']}")
print(f"Mean similarity: {result['similarityDistribution']['mean']}")

Relationships produced: 22064
Nodes compared: 11032
Mean similarity: 0.9576618761486718


In [31]:
gds.run_cypher(
    """
        MATCH (p1:Title)-[r:SIMILAR]->(p2:Title)
        RETURN p1.name AS T1, p2.name AS T2, r.score AS similarity
        ORDER BY similarity DESCENDING, T1, T2
        LIMIT 100
    """
)

,T1,T2,similarity
0,"cleaning, washing, and metal pickling equipment operators and tenders","grinding and polishing workers, hand",1.000000
1,"cleaning, washing, and metal pickling equipment operators and tenders",hoist and winch operators,1.000000
2,"cutting and slicing machine setters, operators, and tenders","derrick operators, oil and gas",1.000000
3,"derrick operators, oil and gas","cutting and slicing machine setters, operators, and tenders",1.000000
4,"first-line supervisors of entertainment and recreation workers, except gambling services",first-line supervisors of personal service workers,1.000000
...,...,...,...
95,"child, family, and school social workers",recycling coordinators,0.964877
96,recycling coordinators,"child, family, and school social workers",0.964877
97,"forestry and conservation science teachers, postsecondary","social work teachers, postsecondary",0.964856
98,butchers and meat cutters,"cutters and trimmers, hand",0.964854


#### **If T1 is similar to T2, then recommended items on T1 is also for T2 (collaborative filtering)**

In [32]:
%%script true

gds.run_cypher(
    """
        MATCH (:Person {name: "Annie"})-[:BUYS]->(p1:Product)
        WITH collect(p1) as products
        MATCH (:Person {name: "Matt"})-[:BUYS]->(p2:Product)
        WHERE not p2 in products
        RETURN p2.name as recommendation
    """
)

Couldn't find program: 'true'


In [33]:
#G.drop()
#gds.graph.drop('TechSkill')

In [34]:
#gds.run_cypher("MATCH (n) DETACH DELETE n")
#gds.graph.drop("full_graph")

In [35]:
gds.close()